In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense, Conv3D, Deconvolution3D, MaxPooling3D, UpSampling3D, Input, Reshape, ZeroPadding3D

- Define the loss function and verify it

In [ ]:
def RadiationLoss(y_true, y_pred):
    """
    Compute the loss for the radiation matrix.
    
    Inputs:
    - y_true: radiation of the target building. 3D Tensor with radiation value at taget surface and others 0.
    - y_pred: the prediction of the radiation.
    
    Returns:
    - scalar mse loss, only calculated where radiation value not equal to zero
    """
    
    y_loc = K.cast(K.not_equal(y_true,K.constant(0)),'float')
    return K.sum(K.pow(y_true-y_pred*y_loc,2))/K.sum(y_loc)
    

In [ ]:
np.random.seed(1)
radiation = np.random.randn(10,10,10,1)
radiation[:5,:5,:5,:] = 0
prediction = np.random.randn(10,10,10,1)
a = K.constant(radiation)
b = K.constant(prediction)
loss = RadiationLoss(a, b)
print('Result:          ', K.eval(loss))
print('Expected result: ', np.sum((radiation-prediction*(radiation!=0))**2)/np.sum(radiation!=0))

- Define the network

In [ ]:
matrix_size = (50, 50, 50, 1)

inp = Input(matrix_size)

# Voxnet structure + autoencoder
enc = Conv3D(32, kernel_size=5, strides=2, padding='same', activation='relu')(inp)
enc = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)

enc = Conv3D(64, kernel_size=5, strides=2, padding='same', activation='relu')(enc)
enc = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)
conv_shape = enc.get_shape().as_list()

enc = Flatten()(enc)
latent = Dense(256, activation='relu')(enc)

dec = Dense(np.prod(conv_shape[1:]), activation='relu')(latent)
dec = Reshape(conv_shape[1:])(dec)

dec = UpSampling3D((2,2,2))(dec)
dec = Deconvolution3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = Deconvolution3D(64, kernel_size=5, strides=2, padding='same', activation='relu')(dec)

dec = UpSampling3D((2,2,2))(dec)
dec = ZeroPadding3D(((0,1),(0,1),(0,1)))(dec) # pad 0s at one side to match the size
dec = Deconvolution3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = Deconvolution3D(32, kernel_size=5, strides=2, padding='same', activation='relu')(dec)

out = Conv3D(1, kernel_size=3, strides=1, padding='same', activation='sigmoid')(dec) # Assume normalized data [0,1]

voxnet_model = Model(inp, out)
voxnet_model.compile(optimizer='adam',loss=RadiationLoss)

voxnet_model.summary()

In [ ]:
X = np.random.rand(128, 50, 50, 50, 1)
y = np.random.rand(128, 50, 50, 50, 1)
y[:,:25,:,:,:] = 0

In [ ]:
history = voxnet_model.fit(X, y, epochs=5)

- Check loss again

In [ ]:
y_pred = voxnet_model.predict(X[:5])
print('Result:          ', voxnet_model.test_on_batch(X[:5],y[:5]))
print('Expected result: ', np.sum((y[:5]-y_pred[:5]*(y[:5]!=0))**2)/np.sum(y[:5]!=0))